# Image PCA & Eigencats Analysis

Principal Component Analysis applied to cat face images, computing eigencats that represent the most significant variations in the dataset.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Utility Functions

Helper functions for visualization and data processing.

In [ ]:
def plot_cats(cats, layout=(2, 2), title=None):
    """Plot multiple cat images in a grid layout.
    
    Args:
        cats: Array of shape (n_images, height, width)
        layout: Tuple of (rows, cols) for subplot grid
        title: Optional title for the figure
    """
    rows, cols = layout
    fig, axes = plt.subplots(rows, cols, figsize=(cols*3, rows*3))
    axes = axes.flatten() if rows * cols > 1 else [axes]
    
    for idx, ax in enumerate(axes):
        if idx < len(cats):
            ax.imshow(cats[idx], cmap='gray')
            ax.axis('off')
        else:
            ax.axis('off')
    
    if title:
        fig.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
def sort_eigvectors(eigenvalues, eigenvectors):
    """Sort eigenvectors by eigenvalues in descending order.
    
    Args:
        eigenvalues: Array of eigenvalues
        eigenvectors: Matrix of eigenvectors
    
    Returns:
        Tuple of (sorted_eigenvalues, sorted_eigenvectors)
    """
    idx = np.argsort(eigenvalues)[::-1]
    return eigenvalues[idx], eigenvectors[:, idx]

## Preprocessing

Calculate the average of all cat images to find the mean cat, then mean-center the dataset. Finally, flatten each image into a vector, resulting in 80 4096-dimensional vectors.

In [ ]:
def preprocess(cats):
    """Preprocess cat images for PCA.
    
    Args:
        cats: Array of shape (n_images, height, width)
    
    Returns:
        Tuple of (mean_cat, centered_flat) where:
        - mean_cat: Mean image (height, width)
        - centered_flat: Mean-centered flattened images (n_images, height*width)
    """
    mean_cat = cats.mean(axis=0)
    centered = cats - mean_cat
    centered_flat = centered.reshape(len(cats), -1)
    return mean_cat, centered_flat

## Eigenvalue Decomposition (EVD)

This section computes the covariance matrix and performs eigendecomposition to find the principal components. Eigenvectors are sorted in descending order by eigenvalues, and the first four eigencats are visualized.

In [ ]:
def eigencats_evd(centered_flat):
    """Compute eigencats using eigenvalue decomposition.
    
    Args:
        centered_flat: Mean-centered flattened images (n_images, n_pixels)
    
    Returns:
        Tuple of (eigenvalues, eigenvectors)
    """
    # Compute covariance matrix
    cov = np.cov(centered_flat.T)
    
    # Eigenvalue decomposition
    eigenvalues, eigenvectors = np.linalg.eig(cov)
    
    # Sort by eigenvalues
    eigenvalues, eigenvectors = sort_eigvectors(eigenvalues, eigenvectors)
    
    return eigenvalues.real, eigenvectors.real

### Example Usage (with placeholder data)

This demonstrates how the functions would be used once we have the actual dataset.

In [ ]:
# Placeholder for future data loading
# cats = np.load('cats.npy')
# mean_cat, centered_flat = preprocess(cats)
# eigenvalues, eigenvectors = eigencats_evd(centered_flat)
# 
# # Visualize first 4 eigencats
# eigencats = eigenvectors[:, :4].T.reshape(4, 64, 64)
# plot_cats(eigencats, layout=(2, 2), title='First 4 Eigencats (EVD)')

## Singular Value Decomposition (SVD)

This section computes eigencats using singular value decomposition, an alternative approach that is often more numerically stable than eigendecomposition.

In [ ]:
def eigencats_svd(cats):    """Compute eigencats using singular value decomposition.        Args:        cats: Array of shape (n_images, height, width)        Returns:        Tuple of (singular_values, eigenvectors)    """    # Mean-center the data    mean_cat = cats.mean(axis=0)    centered = cats - mean_cat    centered_flat = centered.reshape(len(cats), -1)        # Perform SVD    U, S, Vt = np.linalg.svd(centered_flat.T, full_matrices=False)        return S, U

### EVD and SVD Relationship

The singular values from SVD are related to eigenvalues from EVD by the formula: eigenvalues = (singular_values)^2 / (n-1). Both methods should produce equivalent principal components.

## Image Reconstruction

This section reconstructs cat faces using varying numbers of singular values (10, 40, and 80) to demonstrate dimensionality reduction effects.

In [ ]:
def reconstruct_cats(cats, eigenvectors, n_components):
    """Reconstruct images using n_components eigenvectors.
    
    Args:
        cats: Original images (n_images, height, width)
        eigenvectors: Principal components (n_pixels, n_components)
        n_components: Number of components to use
    
    Returns:
        Reconstructed images (n_images, height, width)
    """
    mean_cat = cats.mean(axis=0)
    centered = cats - mean_cat
    centered_flat = centered.reshape(len(cats), -1)
    
    # Project onto principal components and back
    components = eigenvectors[:, :n_components]
    projected = centered_flat @ components
    reconstructed_flat = projected @ components.T
    
    # Add mean back and reshape
    reconstructed = reconstructed_flat.reshape(cats.shape) + mean_cat
    return reconstructed

### Example: Reconstruction with different component counts

Demonstrates the trade-off between compression (fewer components) and reconstruction quality.

In [ ]:
# Placeholder for future reconstruction demo
# for n in [10, 40, 80]:
#     reconstructed = reconstruct_cats(cats, eigenvectors, n)
#     plot_cats(reconstructed[:4], title=f'Reconstructed with {n} components')

### Analysis: Reconstruction Quality vs Compression

With 10 components: High compression but noticeable quality loss. Good for rough identification.

With 40 components: Good balance between compression and quality. Most facial features preserved.

With 80 components: Near-perfect reconstruction using all available components.

## Analysis: Why EVD and SVD Produce Equivalent Results

The EVD and SVD approaches produce equivalent results because they are mathematically related:

- **EVD** operates on the covariance matrix C = X^T X / (n-1), finding eigenvectors and eigenvalues
- **SVD** decomposes the data matrix X directly into X = U Σ V^T
- The right singular vectors V are the eigenvectors of X^T X
- The eigenvalues λ relate to singular values σ by: λ = σ² / (n-1)

Both methods extract the same principal components, just through different computational paths. SVD is often preferred for numerical stability.